# Master course in Object Recognition
## Practice 1

### Title: Deep learning advanced architectures

The goal is to practice advanced deep learning architectures for multi-label classification in [Pascal VOC dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html). We specifically check ResNet50, Inception and MobileNet. We will see 1) how pretrained ResNet50 on imagenet performs on multi-label images, 2) how to modify classification head and 3) implementation of F1 metric.

### NOTES

- Hyperparameters are modifiable,
- The dataset is PASCAL VOC 2012,
- The code uses the KERAS library,
- The code can run in google colab.
- How to finetune on a pretrained model not included (i.e. freeze the pretrained network and train the head, then finetune everything),
- No validation set has been defined. The test and validation sets are the same.

In [1]:
# Imports

import xml.etree.ElementTree as ET
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from keras.layers import (
    Dense,
    GlobalAveragePooling2D,
)
import tensorflow.keras.applications as app
import csv
import time

In [2]:
# Importing from .py files

from metrics import f1_metric
from config import *
from experiment_config import experiments
from train_and_test import train_and_test
from load_data import load_data, create_dataset

In [3]:
train_list = load_data(TRAIN_TXT)
test_list = load_data(TEST_TXT)

# Create dictionaries to store datasets for different batch sizes
train_datasets = {}
test_datasets = {}

start_time = time.time()
# Iterate over batch sizes and create datasets
for batch_size in BATCH_SIZES:
    train_datasets[batch_size] = create_dataset(
        train_list, batch_size, is_training=True
        )
    test_datasets[batch_size] = create_dataset(
        test_list, batch_size, is_training=False
        )
print(f"Time taken to create datasets: {time.time() - start_time} seconds")


Time taken to create datasets: 3.03951358795166 seconds


In [ ]:
# Run default experiments

for exp in experiments["default-experiments"]:
    print(f"Defining model: {exp['title']}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp["net_name"][0]), exp["net_name"][1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp["train_from_scratch"]
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp["warm_up"]

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp["last_layer_activation"])(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp["learning_rate"] * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp["learning_rate"])

    # Compile model
    model.compile(
        loss=exp["loss"],
        optimizer=warmup_optimizer if exp["warm_up"] else opt_rms,
        metrics=["AUC", f1_metric],
    )

    train_dataset = train_datasets[exp["batch_size"]]
    test_dataset = test_datasets[exp["batch_size"]]

    train_and_test(model, exp, train_dataset, test_dataset, train_list, test_list)

In [ ]:
# Determine the best experiment of the 9 default experiments

results_file = RESULTS_CSV

results_data = []

with open(results_file, mode="r") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        if len(row) >= 8:
            results_data.append(row)

highest_test_f1 = -float("inf")
best_id = None

for row in results_data:
    test_f1 = float(row[3])
    if test_f1 > highest_test_f1:
        highest_test_f1 = test_f1
        best_id = int(row[0])

print(best_id)
best_default_experiment_config = next(
    exp for exp in experiments["default-experiments"] if exp["id"] == best_id
)

best_default_experiment_config

In [ ]:
# Run hyperparameter experiments

for exp in experiments["hyperparameter-experiments"]:
    print(f"Defining model: {exp['title']}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp["net_name"][0]), exp["net_name"][1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp["train_from_scratch"]
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp["warm_up"]

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp["last_layer_activation"])(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp["learning_rate"] * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp["learning_rate"])

    # Compile model
    model.compile(
        loss=exp["loss"],
        optimizer=warmup_optimizer if exp["warm_up"] else opt_rms,
        metrics=["AUC", f1_metric],
    )

    train_dataset = train_datasets[exp["batch_size"]]
    test_dataset = test_datasets[exp["batch_size"]]

    train_and_test(model, exp, train_dataset, test_dataset, train_list, test_list)

In [ ]:
# Determine the best experiment of the 9 hyperparameter experiments

results_file = "results.csv"

results_data = []

with open(results_file, mode="r") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        if len(row) >= 8:
            results_data.append(row)

highest_test_f1 = -float("inf")
best_id = None

for row in results_data:
    test_f1 = float(row[3])
    if test_f1 > highest_test_f1:
        highest_test_f1 = test_f1
        best_id = int(row[0])

print(best_id)
best_hyperparameter_experiment_config = next(
    exp for exp in experiments["hyperparameter-experiments"] if exp["id"] == best_id
)

best_hyperparameter_experiment_config

In [ ]:
# Run augmentation experiments

for exp in experiments["augmentation-experiments"]:
    print(f"Defining model: {exp['title']}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp["net_name"][0]), exp["net_name"][1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp["train_from_scratch"]
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp["warm_up"]

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp["last_layer_activation"])(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp["learning_rate"] * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp["learning_rate"])

    # Compile model
    model.compile(
        loss=exp["loss"],
        optimizer=warmup_optimizer if exp["warm_up"] else opt_rms,
        metrics=["AUC", f1_metric],
    )

    train_dataset = train_datasets[exp["batch_size"]]
    test_dataset = test_datasets[exp["batch_size"]]

    train_and_test(model, exp, train_dataset, test_dataset, train_list, test_list)

In [ ]:
# Run classifier head experiments

for exp in experiments["classfier_head_experiments"]:
    print(f"Defining model: {exp['title']}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp["net_name"][0]), exp["net_name"][1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp["train_from_scratch"]
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp["warm_up"]

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp["last_layer_activation"])(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp["learning_rate"] * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp["learning_rate"])

    # Compile model
    model.compile(
        loss=exp["loss"],
        optimizer=warmup_optimizer if exp["warm_up"] else opt_rms,
        metrics=["AUC", f1_metric],
    )

    train_dataset = train_datasets[exp["batch_size"]]
    test_dataset = test_datasets[exp["batch_size"]]

    train_and_test(model, exp, train_dataset, test_dataset, train_list, test_list)